In [14]:
import awswrangler as wr
from transformers import pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
import plotly.express as px

In [15]:
pd.options.mode.copy_on_write = True

In [16]:
df = wr.s3.read_parquet(path="s3://amazon-reviews-eafit/sample/")

In [17]:
df_temp, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_temp, test_size=0.5, random_state=42)

In [18]:
def zero_shot_classifier(
    df: pd.DataFrame,
    size_df=10,
    model="facebook/bart-large-mnli",
) -> None:
    data = df.head(size_df)
    classifier = pipeline(
        "zero-shot-classification",
        model=model,
    )

    results = classifier(
        list(data["review_body"]),
        candidate_labels=["positive", "neutral", "negative"],
    )
    prediction_sentiment = list(map(lambda x: x["labels"][0], results))

    data["prediction_sentiment"] = prediction_sentiment

    accuracy = (data["prediction_sentiment"] == data["sentiment"]).sum() / len(
        data["prediction_sentiment"]
    )

    print(f"The accuracy of the model is: {round(accuracy*100,2)}%")

In [19]:
def sentiment_analysis_classifier(
    df: pd.DataFrame,
    size_df=10,
    model="distilbert-base-uncased-finetuned-sst-2-english",
) -> None:
    data = df.head(size_df)
    data["review_body"] = data["review_body"].apply(lambda x: x[:450])
    classifier = pipeline("sentiment-analysis", model=model)

    results = classifier(list(data["review_body"]))
    prediction_sentiment = list(map(lambda x: x["label"].lower(), results))

    data["prediction_sentiment"] = prediction_sentiment

    accuracy = (data["prediction_sentiment"] == data["sentiment"]).sum() / len(
        data["prediction_sentiment"]
    )

    print(f"The accuracy of the model is: {round(accuracy*100,2)}%")

In [20]:
df_test["review_body_length"] = df_test["review_body"].apply(lambda x: len(x))

In [21]:
fig = px.histogram(
    df_test, x="review_body_length", nbins=1000, title="Review Body Length Distribution"
)
fig.show()

In [22]:
df_test["review_body_length"].describe()

count    199847.000000
mean        335.335792
std         624.104140
min           1.000000
25%          91.000000
50%         161.000000
75%         353.000000
max       44538.000000
Name: review_body_length, dtype: float64

In [23]:
sentiment_analysis_classifier(
    df_test,
    size_df=4000,
)

/home/camilo/proyecto-integrador-semestre-2/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

The accuracy of the model is: 78.38%


In [24]:
zero_shot_classifier(df_test, size_df=4000)

/home/camilo/proyecto-integrador-semestre-2/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



The accuracy of the model is: 84.18%
